   ## CODING PROJECT DBMS FALL 2023

In [ ]:
pip install -U psycopg2

In [ ]:
import psycopg2
from psycopg2 import sql

# We have already created database named database_management.

# Function to execute SQL commands within a transaction
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_SERIALIZABLE

# We have already created database named database_management.
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_SERIALIZABLE

# We have already created database named database_management.


def execute_transaction(sql_commands):
    connection = psycopg2.connect(
        host="localhost",
        database="data_management",
        user="postgres",
        password="Sony@123"
    )
    connection.autocommit = False  # Turn off autocommit to manage transactions
    cursor = connection.cursor()

    try:
        # Set isolation level to SERIALIZABLE
        connection.set_isolation_level(ISOLATION_LEVEL_SERIALIZABLE)

        # Create a savepoint before performing critical operations
        cursor.execute("SAVEPOINT critical_operations")

        for sql_command in sql_commands:
            cursor.execute(sql_command)

        connection.commit()  # Commit the transaction if everything is successful
        print("Query ran successfully")
    except Exception as e:
        print(f"Query Exceution failed: {e}")

        # Rollback to the savepoint if an error occurs during critical operations
        connection.rollback_to_savepoint("critical_operations")
        print("Rolled back to savepoint")

    finally:
        cursor.close()
        connection.close()


In [ ]:
#Run this query in postgresql to create database.
#create database data_management;

In [ ]:
# Drop tables if they already exist
drop_tables = """
DROP TABLE IF EXISTS Stock;
DROP TABLE IF EXISTS Depot;
DROP TABLE IF EXISTS Product;
"""

execute_transaction([drop_tables])



In [ ]:
# Create tables without primary and foreign keys
create_tables = """
CREATE TABLE IF NOT EXISTS Product (
    prodid VARCHAR(10),
    name VARCHAR(25),
    price DECIMAL
);

CREATE TABLE IF NOT EXISTS Depot (
    depid VARCHAR(10),
    addr VARCHAR(30),
    volume DECIMAL
);

CREATE TABLE IF NOT EXISTS Stock (
    prodid VARCHAR(10),
    depid VARCHAR(10),
    quantity DECIMAL
);
"""

# Execute table creation
execute_transaction([create_tables])



Transaction successful


In [ ]:
# Add primary and foreign keys using ALTER TABLE
alter_table_commands = """
ALTER TABLE Product
    ADD PRIMARY KEY (prodid);

ALTER TABLE Product
    ADD CONSTRAINT ck_product_price CHECK (price > 0);

ALTER TABLE Depot
    ADD PRIMARY KEY (depid);

ALTER TABLE Stock
    ADD PRIMARY KEY (prodid, depid);

ALTER TABLE Stock
    ADD CONSTRAINT fk_prodid FOREIGN KEY (prodid) REFERENCES Product (prodid);

ALTER TABLE Stock
    ADD CONSTRAINT fk_stock_depot FOREIGN KEY (depid) REFERENCES Depot (depid);
"""

# Execute ALTER TABLE commands
execute_transaction([alter_table_commands])



Transaction successful


In [ ]:
# Insert initial values
insert_values = """
INSERT INTO Product VALUES
    ('P1', 'Tape', 2.5),
    ('P2', 'TV', 250),
    ('P3', 'Vcr', 80);

INSERT INTO Depot VALUES
    ('d1', 'New York', 9000),
    ('d2', 'Syracuse', 6000),
    ('d4', 'New York', 2000);

INSERT INTO Stock VALUES
    ('P1', 'd1', 1000),
    ('P1', 'd2', -100),
    ('P1', 'd4', 1200),
    ('P3', 'd1', 3000),
    ('P3', 'd4', 2000),
    ('P2', 'd4', 1500),
    ('P2', 'd1', -400),
    ('P2', 'd2', 2000);
"""

# Execute initial value insertion
execute_transaction([insert_values])



Transaction successful


In [ ]:
Transaction = [
"BEGIN;"
#Drop foreign key constraint
"ALTER TABLE Stock DROP CONSTRAINT IF EXISTS fk_prodid;"
#Delete from Stock
"DELETE FROM Stock WHERE prodid = 'P1';"
#Delete from Product
"DELETE FROM Product WHERE prodid = 'P1';"
#Add foreign key constraint back
"ALTER TABLE Stock ADD CONSTRAINT fk_prodid FOREIGN KEY (prodid) REFERENCES Product (prodid) DEFERRABLE INITIALLY IMMEDIATE;"
#"ROLLBACK;"

"COMMIT;"

]

execute_transaction(Transaction)




Transaction successful
